# Computational Performance

## 1. 语言特性  
python 是解释型语言，因此其执行效率低  
Pytorch 为按动态计算图执行  Tensorflow 定义静态计算图后执行，速度更快

In [1]:
import numpy as np
import torch
import dl2_torch as d2l
import math
import torch.nn as nn
import torch.nn.functional as F

import os
import subprocess

d:\Anaconda3\envs\data_torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


计算执行时间的函数

In [2]:
class Benchmark:
    """For measuring running time."""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

In [3]:
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))

In [6]:
net = get_net()
with Benchmark('Without torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)  # 该代码没有提高运行的速度
net = torch.jit.trace(net, x)  # trace适用于没有动态控制流的模型，通常可以提供更稳定的优化
with Benchmark('With torchscript'):
    for i in range(1000): net(x)

Without torchscript: 0.0697 sec
With torchscript: 0.0616 sec


## 2. 异步计算  
异步操作是指任务的发起方（通常是程序中的主线程）在不等待任务完成的情况下继续执行后续操作。当任务执行时，主线程可以继续处理其他事情，一旦任务完成，系统会通知主线程结果或者由主线程处理后续步骤。  
这种模式有助于提高程序的效率和响应速度，特别是在涉及 I/O 操作或计算资源密集的任务中。  

GPU 上首次运行时速度较低，因为需要将数据从 cpu 搬去 gpu， 再次运行发现gpu计算非常快。 

In [11]:
device = d2l.try_gpu()

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = np.random.normal(size=(1000, 1000))
        b = np.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 0.3190 sec
torch: 0.0020 sec


GPU 异步计算实际完成的时间 

In [20]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device)

Done: 0.0871 sec


PyTorch 具有与用户直接交互的前端，例如通过 Python，以及系统用于执行计算的底层。  
前端语言发出的操作被传递到后端执行。后端管理自己的线程，持续收集和执行队列中的任务。请注意，为了实现这一点，后端必须能够跟踪计算图中各个步骤之间的依赖关系。因此，**无法并行化相互依赖**的操作。
<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410101019174.png" width='500px' >
</div>

```python
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z
```

上述代码片段也在图 13.2.2 中进行了说明。每当 Python 前端线程执行前三个语句之一时，它只是将任务返回到后端队列。当需要打印最后语句的结果时，Python 前端线程将等待 C++后端线程完成变量 z 的计算结果。这种设计的一个好处是 Python 前端线程不需要执行实际计算。因此，无论 Python 的性能如何，对程序整体性能的影响都很小。图 13.2.3 说明了前端和后端的交互方式。

<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410101024949.png" width='500px' >
</div>

### 异步阻塞
例如将变量从cuda 移动到 cpu 上时， 需要等待变量计算完成，因此频繁移动计算中间结果会影响计算性能。  

```python
with d2l.Benchmark('synchronous'):
    for _ in range(10000):
        y = x + 1
        y.wait_to_read()

with d2l.Benchmark('asynchronous'):
    for _ in range(10000):
        y = x + 1
    npx.waitall()

synchronous: 3.1623 sec
asynchronous: 0.9288 sec
```

一个简化的 Python 前端线程与 C++后端线程之间的交互可以概括如下：1. 前端命令后端将计算任务 `y = x + 1` 插入队列。1. 后端从队列中接收计算任务并执行实际计算。1. 后端将计算结果返回给前端。  
假设这三个阶段的时间分别为 t1,t2 和 t3 。如果不使用异步编程，执行 10000 次计算的总时间大约为 10000(t1+t2+t3) 。  
如果使用异步编程，执行 10000 次计算的总时间可以减少到 t1+10000t2+t3 （假设 10000t2>9999t1 ），因为前端不需要等待后端返回每次循环的计算结果。

### 其他  
- 深度学习框架可能将 Python 前端与执行后端解耦。这允许快速异步地将命令插入后端及其相关的并行性。
- 异步导致前端响应较快。然而，请注意不要过度填充任务队列，因为它可能导致内存消耗过多。建议为每个小批量同步，以使前端和后端大致同步。

## 3. 自动并行

In [4]:
devices = d2l.try_all_gpus()
def run(x):
    return [x.mm(x) for _ in range(50)]

x_gpu1 = torch.rand(size=(4000, 4000), device=devices[0])

In [5]:
def copy_to_cpu(x, non_blocking=False):
    return [y.to('cpu', non_blocking=non_blocking) for y in x]

Run on GPU1 and copy to CPU: 3.8861 sec


In [7]:
with d2l.Benchmark('Run on GPU1 and copy to CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, True)  # to() copy() 等方法可以通过设置参数为True， 使后端可以异步 计算与转移的操作
    torch.cuda.synchronize()

Run on GPU1 and copy to CPU: 2.4842 sec


In [6]:
with d2l.Benchmark('Run on GPU1 and copy to CPU'):
    y = run(x_gpu1)
    y_cpu = copy_to_cpu(y, False)
    torch.cuda.synchronize()

Run on GPU1 and copy to CPU: 17.1495 sec


<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410110949837.png" width='500px' >
</div>

## 4. 硬件  
https://d2l.ai/chapter_computational-performance/hardware.html  
总线(PCIe)传输大小, 连接CPU GPU SSD Network 等设备。  PCIe 4.0 每条通道能实现双向 16 Gbit/s 的数据传输。  

### CPU  
1. 向量化操作  
深度学习对计算资源需求极大。因此，为了使 CPU 适合机器学习，需要在一次时钟周期内执行许多操作。这通过向量单元实现。  
它们有不同的名称：**在 ARM 上被称为 NEON，在 x86 上（最新一代）被称为 AVX2 单元。一个共同点是它们能够执行 SIMD（单指令多数据）操作**。  
图 13.4.5 展示了在 ARM 上如何在一次时钟周期内将 8 个短整数相加。  
根据架构选择，这些寄存器最长可达 512 位，允许组合多达 64 对数字。例如，我们可能正在乘以两个数字并将它们加到第三个数字上，这也就是所谓的融合乘加操作。  
英特尔的开源 Vino 使用这些操作在服务器级 CPU 上实现深度学习的可观的吞吐量。  
然而，需要注意的是，这个数字完全被 GPU 所能实现的能力所超越。例如，NVIDIA 的 RTX 2080 Ti 拥有 4,352 个 CUDA 核心，每个核心都能在任何时候处理这样的操作。

### GPU  
1. 针对 训练 及 推理 方向的不同优化  
加速器要么针对训练优化，要么针对推理优化。对于后者，我们只需要在网络中计算前向传播。反向传播不需要存储中间数据。此外，我们可能不需要非常精确的计算（通常 FP16 或 INT8 就足够了）。另一方面，在训练过程中，所有中间结果都需要存储以计算梯度。此外，累积梯度需要更高的精度以避免数值下溢（或溢出）。这意味着 FP16（或与 FP32 混合精度）是最低要求。所有这些都要求更快的内存（HBM2 与 GDDR6）和更多的处理能力。例如，NVIDIA 的 Turing T4 GPU 针对推理进行了优化，而 V100 GPU 更适合用于训练。

### 小结  
- 向量化对性能至关重要。确保你了解你加速器的具体能力。例如，一些英特尔至强 CPU 特别适合 INT8 操作，NVIDIA Volta GPU 在 FP16 矩阵-矩阵运算方面表现出色，而 NVIDIA Turing 在 FP16、INT8 和 INT4 操作方面表现出色。  
- 数值溢出由于数据类型过小，在训练过程中可能会成为问题（在推理过程中影响较小）。  
- 别名化会显著降低性能。例如，在 64 位 CPU 上，内存对齐应该以 64 位边界为基准。在 GPU 上，保持卷积大小对齐是一个好主意，例如对齐到张量核心。  
- 将您的算法与硬件相匹配（例如，内存占用和带宽）。当参数适配到缓存中时，可以实现极大的加速（数量级）。 

## 5. GPU 并行

<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410111933964.png" width='800px' >
</div>   

### 数据并行性  
<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410111936989.png" width='800px' >
</div> 

- 在任何训练迭代中，给定一个随机的小批量，我们将批量中的示例分成 k 部分，并将它们均匀地分配到 GPU 上。
- 每个 GPU 根据分配给它的 minibatch 子集计算模型参数的损失和梯度。
- 每个 k GPU 的局部梯度被汇总以获得当前的 minibatch 随机梯度。
- 总体梯度被 **重新分配** 到每个 GPU。
- 每个 GPU 使用这个小批量随机梯度来更新它维护的完整模型参数集。

In [2]:

# Initialize model parameters
scale = 0.01
W1 = torch.randn(size=(20, 1, 3, 3)) * scale
b1 = torch.zeros(20)
W2 = torch.randn(size=(50, 20, 5, 5)) * scale
b2 = torch.zeros(50)
W3 = torch.randn(size=(800, 128)) * scale
b3 = torch.zeros(128)
W4 = torch.randn(size=(128, 10)) * scale
b4 = torch.zeros(10)
params = [W1, b1, W2, b2, W3, b3, W4, b4]

# Define the model
def lenet(X, params):
    h1_conv = F.conv2d(input=X, weight=params[0], bias=params[1])
    h1_activation = F.relu(h1_conv)
    h1 = F.avg_pool2d(input=h1_activation, kernel_size=(2, 2), stride=(2, 2))
    h2_conv = F.conv2d(input=h1, weight=params[2], bias=params[3])
    h2_activation = F.relu(h2_conv)
    h2 = F.avg_pool2d(input=h2_activation, kernel_size=(2, 2), stride=(2, 2))
    h2 = h2.reshape(h2.shape[0], -1)
    h3_linear = torch.mm(h2, params[4]) + params[5]
    h3 = F.relu(h3_linear)
    y_hat = torch.mm(h3, params[6]) + params[7]
    return y_hat

# Cross-entropy loss function
loss = nn.CrossEntropyLoss(reduction='none')

### 数据同步

In [3]:
def get_params(params, device):
    new_params = [p.to(device) for p in params]
    for p in new_params:
        p.requires_grad_()
    return new_params

def allreduce(data):
    for i in range(1, len(data)):
        data[0][:] += data[i].to(data[0].device)
    for i in range(1, len(data)):
        data[i][:] = data[0].to(data[i].device)

### 数据分发

```python
data = torch.arange(20).reshape(4, 5)
devices = [torch.device('cuda:0'), torch.device('cuda:1')]
split = nn.parallel.scatter(data, devices)
print('input :', data)
print('load into', devices)
print('output:', split)
```  

```python
input : tensor([[ 0,  1,  2,  3,  4],  
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
load into [device(type='cuda', index=0), device(type='cuda', index=1)]
output: (tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]], device='cuda:0'), tensor([[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]], device='cuda:1'))
```

In [4]:
def split_batch(X, y, devices):
    """Split `X` and `y` into multiple devices."""
    assert X.shape[0] == y.shape[0]
    return (nn.parallel.scatter(X, devices),
            nn.parallel.scatter(y, devices))

### 训练

In [5]:
def train_batch(X, y, device_params, devices, lr):
    X_shards, y_shards = split_batch(X, y, devices)
    # Loss is calculated separately on each GPU
    ls = [loss(lenet(X_shard, device_W), y_shard).sum()
          for X_shard, y_shard, device_W in zip(
              X_shards, y_shards, device_params)]
    for l in ls:  # Backpropagation is performed separately on each GPU
        l.backward()
    # Sum all gradients from each GPU and broadcast them to all GPUs
    with torch.no_grad():
        for i in range(len(device_params[0])):
            allreduce([device_params[c][i].grad for c in range(len(devices))])
    # The model parameters are updated separately on each GPU
    for param in device_params:
        d2l.sgd(param, lr, X.shape[0]) # Here, we use a full-size batch

In [6]:
def train(num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    devices = [d2l.try_gpu(i) for i in range(num_gpus)]
    # Copy model parameters to `num_gpus` GPUs
    device_params = [get_params(params, d) for d in devices]
    num_epochs = 10
    animator = d2l.Animator('epoch', 'test acc', xlim=[1, num_epochs])
    timer = d2l.Timer()
    for epoch in range(num_epochs):
        timer.start()
        for X, y in train_iter:
            # Perform multi-GPU training for a single minibatch
            train_batch(X, y, device_params, devices, lr)
            torch.cuda.synchronize()
        timer.stop()
        # Evaluate the model on GPU 0
        animator.add(epoch + 1, (d2l.evaluate_accuracy_gpu(
            lambda x: lenet(x, device_params[0]), test_iter, devices[0]),))
    print(f'test acc: {animator.Y[0][-1]:.2f}, {timer.avg():.1f} sec/epoch '
          f'on {str(devices)}')

In [7]:
#  使用单个 gpu 训练
train(num_gpus=1, batch_size=1, lr=0.2)

: 

## 6.GPU 并行简易实现

In [ ]:
def train(net, num_gpus, batch_size, lr):
    train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
    devices = [d2l.try_gpu(i) for i in range(num_gpus)]
    def init_weights(module):
        if type(module) in [nn.Linear, nn.Conv2d]:
            nn.init.normal_(module.weight, std=0.01)
    net.apply(init_weights)
    # Set the model on multiple GPUs
    net = nn.DataParallel(net, device_ids=devices)  # 使用 nn.DataParallel 实现模型的并行计算
    trainer = torch.optim.SGD(net.parameters(), lr)
    loss = nn.CrossEntropyLoss()
    timer, num_epochs = d2l.Timer(), 10
    animator = d2l.Animator('epoch', 'test acc', xlim=[1, num_epochs])
    for epoch in range(num_epochs):
        net.train()
        timer.start()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(devices[0]), y.to(devices[0])  # 将训练数据移至某个设备上
            l = loss(net(X), y)
            l.backward()
            trainer.step()
        timer.stop()
        animator.add(epoch + 1, (d2l.evaluate_accuracy_gpu(net, test_iter),))
    print(f'test acc: {animator.Y[0][-1]:.2f}, {timer.avg():.1f} sec/epoch '
          f'on {str(devices)}')


nn.DataParallel 是 PyTorch 用于将模型并行化以在多个 GPU 上运行的模块。它通过将输入数据拆分到多个设备上，并行计算后再汇总梯度进行更新，达到加速模型训练的目的。

nn.DataParallel 的工作原理  
nn.DataParallel 会自动将数据分配到指定的多个 GPU 上执行计算，然后汇总结果并在主设备上进行梯度更新。

输入拆分：nn.DataParallel 会将一个大的输入批次按 GPU 数量分割成多个小批次，每个 GPU 处理其中一部分。  
模型复制：模型会被复制到每个 GPU 上。在每个设备上运行相同的模型副本，独立处理输入数据并计算梯度。  
结果汇总：各 GPU 独立计算的结果会在主 GPU 上汇总。默认情况下，device_ids[0] 是主设备，梯度更新会在主设备上进行。  

**缺点**：

DataParallel 在某些情况下会有较大的通信开销，因为它需要将数据和梯度在不同 GPU 之间传递。  
如果数据量较大且 GPU 之间的带宽有限，DataParallel 的效率可能不会显著提升，建议使用 torch.nn.parallel.DistributedDataParallel 进行更大规模的分布式训练。

## 7. 并行方式
https://d2l.ai/chapter_computational-performance/parameterserver.html

数据并行的多种方式  
<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410120921259.png" width='800px' >
</div>   

环状同步

多机训练： 使用一个参数服务器收集梯度并对参数进行更新后重新下发  
<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202410120946828.png" width='800px' >
</div>   

## 8. 硬件设备

硬盘数据需要读取到内存，再从内存 到 CPU 到 GPU， 整个数据流带框会受限于 CPU 到 GPU的部分。

<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202408181134751.png" width='800px' >
</div>   

### 1. 多种计算设备

CPU  GPU  DSP(数字信号处理)  FPGA可编程阵列  ASIC专用集成芯片

<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202408181522790.png" width='800px' >
</div>   

### 2. 单机多卡优化

1. **增大 batch_size 通常会将低模型最终的效果**，需要相应的调整lr（在SGD上，可能是随机的优点减弱了？）  

个人理解：fashion-mnist**样本多样性不足**，过大的batchsize使得每次训练拿到的样本特征类似，无法提供更好的训练内容，例如batch中样本相似度高的话，整个梯度会由这些相似的样本决定，相应的模型会更加偏向拟合这个样本（如果样本比例类似，batch较大应该也能提供相当的多样性，不会造成影响），导致训练所得模型精度变低。

课程解释： 假设数据集中所有样本相同，1个batchsize达到收敛需要100次梯度更新， 现在batchsize变10，不改变epoch的情况下梯度更新变为10，而每次计算的梯度与1 batch的时候相同，就需要更多的epoch，或者相对大一点的lr。

2. 多卡训练时，每个GPU上的 batch_size 应该与之前的相同，且应该将 lr 按照倍数左右的比率相应增加。

### 3. 多机多卡训练

https://courses.d2l.ai/zh-v2/assets/pdfs/part-2_4.pdf  
注意调整每个GPU上的训练batchsize，使得训练速度与模型精度达到好的平衡
<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202408181744974.png" width='800px' >
</div>   

tip: **batchsize** 最好为 label数 * [10,20]